# Weights and Biases integration to FinRL
### In this tutorial we are going to integrate Weights and Biases to FinRL along with hyperparameter tuning using sweeps
1. This tutorial is for Stable Baselines3 models 
2. Here we will do hyperparameter optimization using hyperparameter sweeps from Weights and biases
3. Finally, you can jump over to visualizations to pick the best performing hyperparameters
4. The blog post for the tutorial is [here](https://medium.com/analytics-vidhya/weights-and-biases-ify-stable-baselines-models-in-finrl-f11b67f2a6a7)
5. The visualizations and report for the tutorial is [here](https://wandb.ai/athe_kunal/finrl-sweeps-sb3/reports/FinRL-hyperparameter-Sweep--VmlldzoxMTkzNzQ2)

In [1]:
%%capture
!pip install git+https://github.com/AI4Finance-LLC/FinRL-Library.git
!pip install wandb

In [4]:
%%capture
!pip install torch==1.4.0

## Importing packages

In [5]:
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
# matplotlib.use('Agg')
import datetime

%matplotlib inline
from finrl.apps import config
from finrl.finrl_meta.preprocessor.yahoodownloader import YahooDownloader
from finrl.finrl_meta.preprocessor.preprocessors import FeatureEngineer, data_split
from finrl.finrl_meta.env_stock_trading.env_stocktrading import StockTradingEnv
from finrl.finrl_meta.env_stock_trading.env_stocktrading_np import StockTradingEnv as StockTradingEnv_numpy 
# from finrl.drl_agents.stablebaselines3.models import DRLAgent as DRLAgent_sb3
from finrl.finrl_meta.data_processor import DataProcessor

from finrl.plot import backtest_stats, backtest_plot, get_daily_return, get_baseline
import ray
from pprint import pprint
import pprint
import sys
sys.path.append("../FinRL-Library")

import itertools

ImportError: ignored

### Stable Baselines3 callback for W&B

In [ ]:
import wandb
from wandb.integration.sb3 import WandbCallback

## Logging into Weights and Biases
1. First create an account [here](https://wandb.ai/site)
2. Paste the API key to log in 

In [ ]:
wandb.login()

In [ ]:
import os
if not os.path.exists("./" + config.DATA_SAVE_DIR):
    os.makedirs("./" + config.DATA_SAVE_DIR)
if not os.path.exists("./" + config.TRAINED_MODEL_DIR):
    os.makedirs("./" + config.TRAINED_MODEL_DIR)
if not os.path.exists("./" + config.TENSORBOARD_LOG_DIR):
    os.makedirs("./" + config.TENSORBOARD_LOG_DIR)
if not os.path.exists("./" + config.RESULTS_DIR):
    os.makedirs("./" + config.RESULTS_DIR)

## Hyperparameter Space
1. Below we have the hyperparameter search space for PPO, A2C and DDPG.
2. Method determines the search space algorithm and metric is the validation sharpe based on which the next hyperparameter search space is determined
3. Stopping criteria is to discard unpromising trials

In [ ]:
def model_params(model_name):
  sweep_config = {
      'method': 'bayes'
          }

  metric = {
      'name': 'Val sharpe',
      'goal': 'maximize'
  }

  sweep_config['metric'] = metric

  ddpg_param_dict = {
    "buffer_size": {
        "values":[int(1e4), int(1e5), int(1e6)]
        },     
    "learning_rate": {   
        "distribution": "log_uniform",
        "min": 1e-5,
        "max": 1,
    },
    "batch_size" :{
        'values':[32, 64, 128, 256, 512]
    },
  }

  a2c_param_dict = {
      "n_steps": {
          'values': [128, 256, 512, 1024, 2048]},
      "ent_coef": {   
        "distribution": "log_uniform",
        "min": 1e-8,
        "max": 1,
    },
      "learning_rate": {   
        "distribution": "log_uniform",
        "min": 1e-5,
        "max": 1,
    },
  }

  ppo_param_dict = {
      "ent_coef": {   
        "distribution": "log_uniform",
        "min": 1e-8,
        "max": 1,
    },
        "n_steps": {
            'values':[128, 256, 512, 1024, 2048]},
        "learning_rate": {   
        "distribution": "log_uniform",
        "min": 1e-2,
        "max": 1,
    },
        "batch_size": {
        'values':[32, 64, 128, 256, 512]
    },
  }

  stopping_criteria = {'type': 'hyperband', 's': 2, 'eta': 2, 'max_iter': 12}

  sweep_config['early_terminate'] = stopping_criteria

  if model_name == 'ddpg':
    sweep_config['parameters'] = ddpg_param_dict
  elif model_name == 'ppo':
    sweep_config['parameters'] = ppo_param_dict
  elif model_name == 'a2c':
    sweep_config['parameters'] = a2c_param_dict

  return sweep_config

## Custom SB3 Agent to log information into the W&B console

In [ ]:
%%writefile model_wandb.py
import wandb
from wandb.integration.sb3 import WandbCallback
import time

import numpy as np
import pandas as pd
from finrl.apps import config
# from finrl.finrl_meta.env_stock_trading.env_stocktrading import StockTradingEnv
from finrl.finrl_meta.preprocessor.preprocessors import data_split
from stable_baselines3 import A2C, DDPG, PPO, SAC, TD3
from stable_baselines3.common.callbacks import BaseCallback
from stable_baselines3.common.noise import (
    NormalActionNoise,
    OrnsteinUhlenbeckActionNoise,
)
from stable_baselines3.common.vec_env import DummyVecEnv
import pprint
MODELS = {"a2c": A2C, "ddpg": DDPG, "td3": TD3, "sac": SAC, "ppo": PPO}

MODEL_KWARGS = {x: config.__dict__[f"{x.upper()}_PARAMS"] for x in MODELS.keys()}

NOISE = {
    "normal": NormalActionNoise,
    "ornstein_uhlenbeck": OrnsteinUhlenbeckActionNoise,
}
 
class DRLAgent_SB3:
  def __init__(self,env,run):
    self.env = env
    # self.run = wandb.init(reinit=True,
    #       project = 'finrl-sweeps-sb3',
    #       sync_tensorboard = True,
    #       save_code = True
    #   )
    self.run = run
  def get_model(
      self,
      model_name,
      policy_kwargs=None,
      model_kwargs=None,
      verbose=1,
      seed=None,
  ):
      if model_name not in MODELS:
          raise NotImplementedError("NotImplementedError")

      if model_kwargs is None:
          model_kwargs = MODEL_KWARGS[model_name]

      if "action_noise" in model_kwargs:
          n_actions = self.env.action_space.shape[-1]
          model_kwargs["action_noise"] = NOISE[model_kwargs["action_noise"]](
              mean=np.zeros(n_actions), sigma=0.1 * np.ones(n_actions)
          )
      print(model_kwargs)

      model = MODELS[model_name](
          policy='MlpPolicy',
          env=self.env,
          tensorboard_log=f"runs/{self.run.id}",
          verbose=verbose,
          policy_kwargs=policy_kwargs,
          seed=seed,
          **model_kwargs,
      )
      return model
  
  def train_model(self, model,total_timesteps):
    model = model.learn(
        total_timesteps=total_timesteps,
        callback = WandbCallback(
            gradient_save_freq = 100, model_save_path = f"models/{self.run.id}",
            verbose = 2
        ),
    )
    
    return model
  @staticmethod
  def DRL_prediction_load_from_file(run , model_name, environment,val_or_test='val'):
      if model_name not in MODELS:
          raise NotImplementedError("NotImplementedError, Pass correct model name")
      try:
          # load agent
          model = MODELS[model_name].load(f"models/{run.id}/model.zip") #print_system_info=True
          print("Successfully load model", f"models/{run.id}")
      except BaseException:
          raise ValueError("Fail to load agent!")

      # test on the testing env
      state = environment.reset()
      episode_returns = list()  # the cumulative_return / initial_account
      episode_total_assets = list()
      episode_total_assets.append(environment.initial_total_asset)
      done = False
      while not done:
          action = model.predict(state)[0]
          state, reward, done, _ = environment.step(action)

          total_asset = (
              environment.amount
              + (environment.price_ary[environment.day] * environment.stocks).sum()
          )
          episode_total_assets.append(total_asset)
          episode_return = total_asset / environment.initial_total_asset
          episode_returns.append(episode_return)
    
      def calculate_sharpe(df):
        df['daily_return'] = df['account_value'].pct_change(1)
        if df['daily_return'].std() !=0:
          sharpe = (252**0.5)*df['daily_return'].mean()/ \
              df['daily_return'].std()
          return sharpe
        else:
          return 0

      print("episode_return", episode_return)
      print("Test Finished!")
      sharpe_df = pd.DataFrame(episode_total_assets,columns=['account_value'])
      sharpe = calculate_sharpe(sharpe_df)
      if val_or_test == "val":
        wandb.log({"Val sharpe":sharpe})
      elif val_or_test == "test":
        wandb.log({"Test sharpe":sharpe})
        print(f'Test Sharpe for {run.id} is {sharpe}')
      # run.finish()
      return sharpe, episode_total_assets

In [ ]:
from model_wandb import DRLAgent_SB3

## Getting the training environment and train function
1. In the training function we pick the hyperparameter config and train the model followed by saving
2. Then the trained model is used to calculate the validation sharpe
3. The validation sharpe is logged into the console and as per Bayes theorem the next search space is selected to improve Val Sharpe ratio

In [ ]:
def train_agent_env(start_date, end_date, ticker_list, data_source, time_interval, 
          technical_indicator_list, env, model_name, if_vix = True,
          **kwargs):
    
    #fetch data
    DP = DataProcessor(data_source, **kwargs)
    DP.download_data(ticker_list, start_date, end_date, time_interval)
    DP.clean_data()
    DP.add_technical_indicator(technical_indicator_list)
    if if_vix:
        DP.add_vix()
    # data.to_csv('train_data.csv')
    # data = pd.read_csv('train_data.csv')
    price_array, tech_array, turbulence_array = DP.df_to_array(if_vix)
    env_config = {'price_array':price_array,
              'tech_array':tech_array,
              'turbulence_array':turbulence_array,
              'if_train':True}
    env_instance = env(config=env_config)

    return env_instance

def train(config=None):
    with wandb.init(config=config, sync_tensorboard = True, save_code = True) as run:
      #Get the training environment
      train_env_instance = train_agent_env(TRAIN_START_DATE, TRAIN_END_DATE, ticker_list, data_source, time_interval, 
                            technical_indicator_list, env, model_name)
      config = wandb.config
      #Initialize the training agent
      agent_train = DRLAgent_SB3(train_env_instance,run)
      #For current set of hyperparameters initialize the model
      model = agent_train.get_model(model_name, model_kwargs = config)
      #train the model
      trained_model = agent_train.train_model(model,total_timesteps)
      run_ids[run.id] = run
      print('Training finished!')
      #Log the validation sharpe
      sharpe,val_episode_total_asset = val_or_test(
          VAL_START_DATE, VAL_END_DATE,run,ticker_list, 
          data_source, time_interval, 
          technical_indicator_list, env, model_name
      )
      #Log the testing sharpe
      sharpe,val_episode_total_asset = val_or_test(
          TEST_START_DATE, TEST_END_DATE,run,ticker_list, 
          data_source, time_interval, 
          technical_indicator_list, env, model_name,val_or_test = 'test'
      )
     

## Validation or Testing function

In [ ]:
def val_or_test(start_date, end_date,run, ticker_list, data_source, time_interval, 
         technical_indicator_list, env, model_name,val_or_test='val', if_vix = True,
         **kwargs):
  
  DP = DataProcessor(data_source, **kwargs)
  DP.download_data(ticker_list, start_date, end_date, time_interval)
  DP.clean_data()
  DP.add_technical_indicator(technical_indicator_list)
  
  if if_vix:
      DP.add_vix()
  # if val_or_test == 'val':
  #   data.to_csv('val.csv')
  # elif val_or_test == 'test':
  #   data.to_csv('test.csv')
  # if val_or_test == 'val':
  #   data=pd.read_csv('val.csv')
  # elif val_or_test == 'test':
  #   data = pd.read_csv('test.csv')
  price_array, tech_array, turbulence_array = DP.df_to_array(if_vix)
    
  test_env_config = {'price_array':price_array,
          'tech_array':tech_array,
          'turbulence_array':turbulence_array,
          'if_train':False}
  env_instance = env(config=test_env_config)
  
  run_ids[run.id] = run
  sharpe,episode_total_assets = DRLAgent_SB3.DRL_prediction_load_from_file(run,model_name,env_instance,val_or_test)
  
  return sharpe, episode_total_assets

## Defining the variables

In [ ]:
TRAIN_START_DATE = '2009-01-01'
TRAIN_END_DATE = '2019-07-30'

VAL_START_DATE = '2019-08-01'
VAL_END_DATE = '2020-07-30'

TEST_START_DATE = '2020-08-01'
TEST_END_DATE = '2021-10-01'

# ticker_list = config.DOW_30_TICKER
ticker_list = ['TSLA']
data_source = 'yahoofinance'
time_interval = '1D'
technical_indicator_list = config.TECHNICAL_INDICATORS_LIST
env = StockTradingEnv_numpy
model_name = "a2c"

# PPO_PARAMS = {
#     "n_steps": 2048,
#     "ent_coef": 0.01,
#     "learning_rate": 0.00025,
#     "batch_size": 128,
# }

total_timesteps = 15000
run_ids = {}


## Running the hyperparameter sweep for our model
1. Max initial failures ensures that our agent does not stop after these number of minimum trials
2. Number of trials is set to 30, so it does hyperparameter search for 30 times

In [ ]:
count = 30
os.environ['WANDB_AGENT_MAX_INITIAL_FAILURES']= str(count-5)
project_name = 'finrl-sweeps-sb3'
sweep_config = model_params(model_name)

sweep_id = wandb.sweep(sweep_config,project=project_name)
wandb.agent(sweep_id, train, count=count)

In [ ]:
run_ids

## End of the Tutorial
1. Go and checkout the visualization of the hyperparamter sweep in your wandb console or you check it [here](https://wandb.ai/athe_kunal/finrl-sweeps-sb3/reports/FinRL-hyperparameter-Sweep--VmlldzoxMTkzNzQ2)